# Web Mining and Applied NLP (44-620)

## Final Project: Article Summarizer

### Student Name: David Rodriguez-Mayorquin

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

You should bring in code from previous assignments to help you answer the questions below.

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

**Question 1**

 Find on the internet an article or blog post about a topic that interests you and you are able to get the text for using the technologies we have applied in the course.  Get the html for the article and store it in a file (which you must submit with your project)

I used the Spotify API to retrieve saved tracks from my own Spotify profile. The data was returned as JSON, which I processed  and converted into HTML file (saved_tracks.html). The file displays my saved Spotify tracks with clickable links to the Spotify pages for each song.

**Question 2** 

Read in your article's html source from the file you created in question 1 and do sentiment analysis on the article/post's text (use `.get_text()`).  Print the polarity score with an appropriate label.  Additionally print the number of sentences in the original article (with an appropriate label)

In [7]:
import pickle
import re
from bs4 import BeautifulSoup
from textblob import TextBlob

# Load the pickle file
with open("spotify_tracks.pkl", "rb") as pickle_file:
    html_content = pickle.load(pickle_file)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
article_text = soup.get_text()

# Print first 1000 characters of extracted text for debugging
print("Extracted Text (First 1000 characters):")
print(repr(article_text[:1000]))

# Refined regex for splitting text into sentences
sentences = re.split(r'(?:\n{2,}|\n\s*|\s{2,}|(?<=[.!?]) +)', article_text)  # Split at newlines, double spaces, or punctuation
sentences = [sentence.strip() for sentence in sentences if sentence.strip()]  # Remove empty sentences

# Count the number of sentences
num_sentences = len(sentences)

# Perform sentiment analysis on the entire text
blob = TextBlob(article_text)
polarity_score = blob.sentiment.polarity  # Polarity ranges from -1 (negative) to 1 (positive)

# Print results
print("\nExtracted Sentences (First 10):", sentences[:10])  # Show the first 10 sentences for verification
print("Number of Sentences (Refined Split):", num_sentences)
print("\nPolarity Score:", polarity_score)


Extracted Text (First 1000 characters):
"Spotify Tracks\n\nTrack: Soy Sólo Un Secreto\nAlbum: Fuerza\nArtist: Alejandra Guzman\n    \n\nTrack: Time (Clock Of The Heart) - Remastered 2003\nAlbum: Culture Club\nArtist: Culture Club\n    \n\nTrack: Words - Original Version 1983\nAlbum: Words\nArtist: F.R. David\n    \n\nTrack: Leather Jacket\nAlbum: Leather Jacket\nArtist: The Vantages\n    \n\nTrack: Knowing Me Knowing You\nAlbum: Volumes\nArtist: MGT\n    \n\nTrack: Circle In The Sand\nAlbum: Heaven On Earth (Deluxe Edition)\nArtist: Belinda Carlisle\n    \n\nTrack: When Love Calls Your Name\nAlbum: Love Hurts\nArtist: Cher\n    \n\nTrack: I'll Be There Where The Heart Is\nAlbum: Flashdance (Original Soundtrack From The Motion Picture)\nArtist: Kim Carnes\n    \n\nTrack: CHIHIRO\nAlbum: HIT ME HARD AND SOFT\nArtist: Billie Eilish\n    \n\nTrack: What I Am\nAlbum: Shooting Rubberbands At The Stars\nArtist: Edie Brickell & New Bohemians\n    \n\nTrack: I Melt with You\nAlbum: After the Sn

The sentiment polarity score of 0.0232 suggests a slightly positive tone overall.

**Question3**

Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent tokens (converted to lower case).  Print the common tokens with an appropriate label.  Additionally, print the tokens their frequencies (with appropriate labels)

In [8]:
import spacy
from bs4 import BeautifulSoup
from collections import Counter

# Read and extract text from HTML
with open("saved_tracks.html", "r", encoding="utf-8") as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, "html.parser")
text = soup.get_text()

# Load spaCy pipeline
nlp = spacy.load("en_core_web_sm")  # Ensure this line is executed
doc = nlp(text)  # Process the text

# Tokenize and filter valid tokens
tokens = [
    token.text.lower()
    for token in doc
    if not token.is_stop and not token.is_punct and token.text.strip()
]

# Count token frequencies
token_counts = Counter(tokens)

# Get the 5 most common tokens
common_tokens = token_counts.most_common(5)

# Print results
print("Top 5 Most Frequent Tokens:")
for token, freq in common_tokens:
    print(f"Token: '{token}', Frequency: {freq}")


Top 5 Most Frequent Tokens:
Token: 'spotify', Frequency: 2
Token: 'tracks', Frequency: 2
Token: 'heart', Frequency: 2
Token: 'knowing', Frequency: 2
Token: 'saved', Frequency: 1


4. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent lemmas (converted to lower case).  Print the common lemmas with an appropriate label.  Additionally, print the lemmas with their frequencies (with appropriate labels).

5. Make a list containing the scores (using tokens) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores. From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

6. Make a list containing the scores (using lemmas) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores.  From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

7. Using the histograms from questions 5 and 6, decide a "cutoff" score for tokens and lemmas such that fewer than half the sentences would have a score greater than the cutoff score.  Record the scores in this Markdown cell

* Cutoff Score (tokens): 
* Cutoff Score (lemmas):

Feel free to change these scores as you generate your summaries.  Ideally, we're shooting for at least 6 sentences for our summary, but don't want more than 10 (these numbers are rough estimates; they depend on the length of your article).

8. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on tokens) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

9. Print the polarity score of your summary you generated with the token scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

10. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on lemmas) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

11. Print the polarity score of your summary you generated with the lemma scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

12.  Compare your polarity scores of your summaries to the polarity scores of the initial article.  Is there a difference?  Why do you think that may or may not be?.  Answer in this Markdown cell.  

13. Based on your reading of the original article, which summary do you think is better (if there's a difference).  Why do you think this might be?